In [1]:
import pandas as pd

# Load data
reviews_df = pd.read_json('../../data/raw/yelp_review.json', lines=True)
users_df = pd.read_json('../../data/raw/yelp_user.json', lines=True)
business_df = pd.read_json('../../data/raw/yelp_business.json', lines=True)